# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using Turi Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Turi Create

In [1]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
sales = turicreate.SFrame('kc_house_data.csv')

Finished parsing file /home/astha/kc_house_data.csv

Parsing completed. Parsed 100 lines in 0.325953 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,float,int,float,int,int,float,int,int,int,int,int,int,int,int,int,float,float,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/astha/kc_house_data.csv

Parsing completed. Parsed 21613 lines in 0.14692 secs.

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [4]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [5]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in Turi Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [6]:
model_all = turicreate.linear_regression.create(sales, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.631902     | 6869667.894269     | 385998.458664                   |

| 10        | 11       | 0.000002  | 1.814834     | 6842123.230167     | 364204.576307                   |

+-----------+----------+-----------+--------------+--------------------+---------------------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [24]:
print(model_all.coefficients['value'].nnz())
coefficient = model_all.coefficients
value = coefficient["value"]
name = coefficient["name"]
for i in range(len(value)):
    if value[i] != 0.0:
        print(name[i])

6
(intercept)
bathrooms
sqft_living
sqft_living_sqrt
grade
sqft_above


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [25]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [33]:
import numpy as np
rss = []
for l1_penalty in np.logspace(1, 7, num=13):
    model = turicreate.linear_regression.create(training, target='price', features=all_features, l1_penalty=l1_penalty, l2_penalty=0, validation_set= None, verbose = False)
    predictions = model.predict(validation)
    errors = predictions - validation['price']
    RSS = sum(errors**2)
    rss.append(RSS)

In [34]:
print(rss.index(min(rss)), min(rss))

0 625766294657733.5


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

first value of RSS is least so best l1_penalty is 10.

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [37]:
model = turicreate.linear_regression.create(training, target='price', features=all_features, l1_penalty=10, l2_penalty=0, validation_set= None, verbose = False)
print('number of non-zero weights:',model.coefficients['value'].nnz())

number of non-zero weights: 18


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [38]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [39]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model.coefficients['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [40]:
for l1_penalty in l1_penalty_values:
    model = turicreate.linear_regression.create(training, features=all_features, target='price', l1_penalty=l1_penalty, validation_set=None, l2_penalty=0, verbose=False)
    print('Number of non-zeros: ',model.coefficients['value'].nnz())
    

Number of non-zeros:  18
Number of non-zeros:  18
Number of non-zeros:  18
Number of non-zeros:  18
Number of non-zeros:  17
Number of non-zeros:  17
Number of non-zeros:  17
Number of non-zeros:  17
Number of non-zeros:  17
Number of non-zeros:  16
Number of non-zeros:  15
Number of non-zeros:  15
Number of non-zeros:  13
Number of non-zeros:  12
Number of non-zeros:  10
Number of non-zeros:  6
Number of non-zeros:  5
Number of non-zeros:  3
Number of non-zeros:  1
Number of non-zeros:  1


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [41]:
l1_penalty_values

array([1.00000000e+08, 1.27427499e+08, 1.62377674e+08, 2.06913808e+08,
       2.63665090e+08, 3.35981829e+08, 4.28133240e+08, 5.45559478e+08,
       6.95192796e+08, 8.85866790e+08, 1.12883789e+09, 1.43844989e+09,
       1.83298071e+09, 2.33572147e+09, 2.97635144e+09, 3.79269019e+09,
       4.83293024e+09, 6.15848211e+09, 7.84759970e+09, 1.00000000e+10])

In [42]:
l1_penalty_min = l1_penalty_values[-6]
l1_penalty_max = l1_penalty_values[-5]

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

In [43]:
print('minimum l1 penalty:', l1_penalty_min)
print('maximum l1 penalty:', l1_penalty_max)

minimum l1 penalty: 2976351441.6313133
maximum l1 penalty: 3792690190.7322536


## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [44]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [64]:
rss_max_nonzeros = []
for l1_penalty in l1_penalty_values:
    model = turicreate.linear_regression.create(training, target='price', features=all_features, l1_penalty=l1_penalty, l2_penalty=0, validation_set=None, verbose=False)
    predictions = model.predict(validation)
    errors = validation['price'] - predictions
    RSS = sum(errors**2)
    if model.coefficients['value'].nnz() == max_nonzeros:
        rss_max_nonzeros.append(RSS)
        print('sparsity:', model.coefficients['value'].nnz())
        print('l1 penalty:', l1_penalty, '\nRSS:',RSS)

sparsity: 7
l1 penalty: 3448968612.163437 
RSS: 1046937489533986.4
sparsity: 7
l1 penalty: 3491933809.484539 
RSS: 1051147625263915.5
sparsity: 7
l1 penalty: 3534899006.805641 
RSS: 1055992734935845.2
sparsity: 7
l1 penalty: 3577864204.126743 
RSS: 1060799531527683.1


In [65]:
print(rss_max_nonzeros.index(min(rss_max_nonzeros)))
print('lowest RSS:', min(rss_max_nonzeros))

0
lowest RSS: 1046937489533986.4


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [70]:
model = turicreate.linear_regression.create(training, target='price', features=all_features, l1_penalty = 3448968612.163437, l2_penalty=0, validation_set=None, verbose=False)
print(model.coefficients)
coefficient = model.coefficients
value = coefficient["value"]
name = coefficient["name"]
print('\nnon-zero coefficients-\n')
for i in range(len(value)):
    if value[i] != 0.0:
        print(name[i])

+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 222253.19177946472 |  None  |
|     bedrooms     |  None | 661.7225625501316  |  None  |
| bedrooms_square  |  None |        0.0         |  None  |
|    bathrooms     |  None | 15873.957390454756 |  None  |
|   sqft_living    |  None | 32.41022208406935  |  None  |
| sqft_living_sqrt |  None | 690.1147756379333  |  None  |
|     sqft_lot     |  None |        0.0         |  None  |
|  sqft_lot_sqrt   |  None |        0.0         |  None  |
|      floors      |  None |        0.0         |  None  |
|  floors_square   |  None |        0.0         |  None  |
+------------------+-------+--------------------+--------+
[18 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

non-zero coefficients-


* Lowest l1 penalty in our validation set having sparsity equal to max_nonzeros is 3448968612.163437
* Features having non-zero coefficients
1. bedrooms
2. bathrooms
3. sqft_living
4. sqft_living_sqrt
5. grade
6. sqft_above